In [80]:
css = '''<style>
@import url(https://fonts.googleapis.com/css?family=Montserrat);
* {
  margin: 0;
  padding: 0;
  font-family: "Montserrat", sans-serif;
  /*   outline: 1px solid white; */
}

body {
  background: #000000;
  height: 100vh;
  width: 100%;
  display: flex;
  padding-top: 1rem;
}
.base-container {
  width: 100%;
}
.friend-text-div {
  display: flex;
  margin-left: 0.5rem;
}

.friend-text-div > img {
  height: 2rem;
  align-self: flex-end;
}

.friend-text-container {
  width: 10rem;
  display: flex;
  flex-direction: column;
}

.friend-text {
  background: #252525;
  border-radius: 0.5rem;
  color: #fff;
  height: fit-content;
  width: fit-content;
  padding: 0.5rem 1rem;
  margin: 0.12rem 0.5rem;
}

.my-text-div {
  display: flex;
  justify-content: flex-end;
}

.my-text-div > img {
  height: 2rem;
  align-self: flex-end;
}

.my-text-container {
  display: flex;
  flex-direction: column;
  align-items: flex-end;
}

.my-text {
  background: linear-gradient(
      180deg,
      rgba(138, 46, 183, 1) 1%,
      rgba(101, 89, 206, 1) 52%,
      rgba(88, 115, 220, 1) 91%
    )
    no-repeat center;
  background-attachment: fixed;
  color: #fff;
  border-radius: 0.5rem 0.2rem 0.2rem 0.5rem;
  height: fit-content;
  width: fit-content;
  padding: 0.5rem 1rem;
  margin: 0.12rem 0.5rem;
}

.my-text-container > div:first-child {
  border-radius: 0.5rem 1rem 0.2rem 0.5rem;
}
.my-text-container > div:last-child {
  border-radius: 0.5rem 0.2rem 1rem 0.5rem;
}
.friend-text-container > div:first-child {
  border-radius: 1rem 0.5rem 0.2rem 0.5rem;
}
.friend-text-container > div:last-child {
  border-radius: 0.5rem 0.2rem 0.5rem 1rem;
}
</style>'''

html = f'''
<html>
<head>
{css}
</head>
<body>
<divs/>
<body>
</body>
</html>
'''

div_base = '''
<div class="base-container">
<conversation/>
</div>
'''

div_friend = '''
  <div class="friend-text-div">
    <img src='https://avataaars.io/?avatarStyle=Circle&topType=LongHairStraight&accessoriesType=Blank&hairColor=BrownDark&facialHairType=Blank&clotheType=BlazerShirt&eyeType=Default&eyebrowType=Default&mouthType=Default&skinColor=Light'
/>
    <div class="friend-text-container">
      <div class="friend-text"><txt/></div>
    </div>
  </div>
'''

div_my = '''
  <div class="my-text-div">
    <div class="my-text-container">
      <div class="my-text"><txt/></div>
    </div>
    <img src='https://avataaars.io/?avatarStyle=Circle&topType=LongHairMiaWallace&accessoriesType=Round&hairColor=Blonde&facialHairType=Blank&clotheType=ShirtCrewNeck&clotheColor=Gray02&eyeType=Happy&eyebrowType=Default&mouthType=Default&skinColor=Light'
/>
  </div>
'''


In [15]:
# def process_comment(comment):
#     comment = comment.strip().replace('「', '').replace('」', '').replace('：', ':')
#     if comment[0] == 'A':
#         return ('A', comment.replace('A:', '').strip())
#     elif comment[0] == 'B':
#         return ('B', comment.replace('B:', '').strip())
#     else:
#         return ('other', comment)

In [78]:
# def create_conversation_div(comments):
#     person_divs = []
#     for comment in comments:
#         if comment[0] == 'A':
#             person_divs.append(div_friend.replace('<txt/>', comment[1]))
#         elif comment[0] == 'B':
#             person_divs.append(div_my.replace('<txt/>', comment[1]))
#         else:
#             person_divs.append(div_my.replace('<txt/>', comment[1])) # 後で変える
#     person_txt = ''.join(person_divs) + '\n'
#     divs = div_base.replace('<conversation/>', person_txt)
#     return html.replace('<divs/>', divs)

def create_conversation_div(comments, speaker1, speaker2):
    person_divs = []
    for comment in comments:
        if comment['speaker'] == speaker1:
            person_divs.append(div_friend.replace('<txt/>', comment['comment']))
        elif comment['speaker'] == speaker2:
            person_divs.append(div_my.replace('<txt/>', comment['comment']))
        else:
            person_divs.append(div_my.replace('<txt/>', comment['comment']))
    person_txt = ''.join(person_divs) + '\n'
    divs = div_base.replace('<conversation/>', person_txt)
    return html.replace('<divs/>', divs)

In [ ]:
# https://saasis.jp/2023/03/06/%E9%81%82%E3%81%AB%E5%85%AC%E9%96%8B%E3%81%95%E3%82%8C%E3%81%9Fchatgpt-api%E3%81%A8%E3%81%AF%EF%BC%9F-%E5%88%A9%E7%94%A8%E3%81%99%E3%82%8B%E3%81%BE%E3%81%A7%E3%81%AE%E6%B5%81%E3%82%8C%E3%80%90/

import openai
import json

def get_openai_apikey():
    path = r'E:\secrets\openai_apikey.txt'
    with open(path, 'r') as f:
        return f.read().replace('\n', '')

def get_chatgpt_content(keyword, speaker1, speaker2):
    openai.api_key = get_openai_apikey()

    content = '''# 指示
    ウィキペディアから「{0}」を検索して、その内容を{1}と{2}という二人の女性が質問と回答を繰り返す対話形式で説明してください。
    挨拶は不要です。敬語は不要です。ですます調は不要です。二人は親友のように会話します。出力はjson形式にしてください。

    # json形式
    [
        {{"speaker": 女性, "comment": 会話内容}},
        {{"speaker": 女性, "comment": 会話内容}}
    ]

    # 出力
    '''.format(keyword, speaker1, speaker2)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": content},
        ]
    )
    jsontxt = response["choices"][0]["message"]["content"]
    return json.loads(jsontxt)


keyword = '応仁の乱'
speaker1 = '明日香'
speaker2 = '麻衣'
# content = get_chatgpt_content(keyword)
# comments = [process_comment(c) for c in content.split('\n\n') if c.strip() != '']
comments = get_chatgpt_content(keyword, speaker1, speaker2)
conv = create_conversation_div(comments, speaker1, speaker2)
print(conv.replace('\n\n', '\n'))

In [82]:
openai.api_key = get_openai_apikey()

content = '''# 指示
ウィキペディアから「{0}」を検索して、その内容を{1}と{2}という二人の女性が質問と回答を繰り返す対話形式で説明してください。
挨拶は不要です。敬語は不要です。ですます調は不要です。二人は親友のように会話します。出力はjson形式にしてください。

# json形式
[
    {{"speaker": 女性, "comment": 会話内容}},
    {{"speaker": 女性, "comment": 会話内容}}        
]

# 出力
'''.format(keyword, speaker1, speaker2)

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": content},
    ]
)
jsontxt = response["choices"][0]["message"]["content"]

In [208]:
import random

with open('articles.csv', 'r') as f:
    lines = f.readlines()
lines = lines[1:] # remove the header
line = random.choice(lines).strip()
dict(title=line.split(',')[1], url=line.split(',')[2])


{'title': '国勢調査で宗教を「ジェダイ」と回答する動き',
 'url': 'https://ja.wikipedia.org/wiki/%E5%9B%BD%E5%8B%A2%E8%AA%BF%E6%9F%BB%E3%81%A7%E5%AE%97%E6%95%99%E3%82%92%E3%80%8C%E3%82%B8%E3%82%A7%E3%83%80%E3%82%A4%E3%80%8D%E3%81%A8%E5%9B%9E%E7%AD%94%E3%81%99%E3%82%8B%E5%8B%95%E3%81%8D'}

In [1]:
import time
import pyautogui as pgui

In [3]:
pgui.position()

# Point(x=689, y=480)
# Point(x=634, y=670)

Point(x=634, y=670)

In [5]:
for _ in range(72):
    time.sleep(600) # 10分待機
    pgui.click(x=689, y=480, duration=1) # 指定した座標に1秒かけて移動してクリック
    time.sleep(1) # 1秒待機
    pgui.click(x=634, y=670, duration=1)